In [18]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [19]:
rayfile = "/home/anuran/repos/Vulcan/bin/rayDump.rays"

In [20]:
# load rays
rays_struct = np.dtype(
    [
        ('sx', np.float32),
        ('sy', np.float32),
        ('ex', np.float32),
        ('ey', np.float32),
        ('r' , np.float32),
        ('th', np.float32),
        ('i',  np.uint16),
        ('idx',  np.uint16),
        ('pad',np.int32),
        ('ts', np.float64), #remove for old ray file formats
      ])

# rays = np.memmap(rayfile,rays_struct, 'r',1)
rays0 = np.fromfile(rayfile,rays_struct)
rays0 = rays0[:int(len(rays0)//1081*1081)]

In [4]:
# split rays by sensor THIS IS A HACK RIGHT NOW!
# .... actually, don't bother

In [33]:
#Debug Cell
%matplotlib widget
# quickly plot the bot path and rays
substart=0
subend=int(2.0e9)
substart=int(substart//1081)*1081
subend=int(subend//1081)*1081
plt.plot(rays0['ex'][substart:subend][::],rays0['ey'][substart:subend][::1],',')
plt.plot(rays0['sx'][substart:subend][::997],rays0['sy'][substart:subend][::997],',')
print(len(rays0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

31904634


In [14]:
#HACK use only one pass for algo debug
# rays0 =  rays0[substart:subend]
# rays0 = rays0[:int(len(rays0)//1081*1081)]

# Hyperparameter Setting Instructions
1. CELLS_PER_M should indicate a cell width less than the max displacement of the robot between scans. 
   1. Otherwise there may be gaps between adjacent rays in the same direction.
2. Theta width should be >= laser angular spacing, and <= min peak half-width at worst laser plane tilt
   1. If you can't satisfy both, mount your laser better, stiffen your suspension, or get a blurrier (higher divergence) laser!
3. Max range should be set so max_turn_rate * max_range < cell_width
4. Currently, the max distance is hardcoded at 200 cells


In [32]:
## Find the max extent of the map and transforms

keep=(rays0['r']<10000)*(rays0['r']<10000)
rays=rays0[keep==1]
## Figure out rasterization
# Floor of float coords is integer coords
# Images are y flipped

def homp(T,pts):
    tmppts = pts @ T[:-1,:-1].T + T[:-1,-1]
    denom = pts @ T[-1:,:-1].T + T[-1,-1]
    return tmppts/denom


CELLS_PER_M = 20.0
print("Cell width:", 1/CELLS_PER_M)

formatter = {'float':lambda x:np.format_float_positional(x,precision=2,fractional=True,trim='-',pad_left=8,pad_right=2)}

def get_T_px_f_with_extent(rays, debug=False):
    # The corners in world space
    f_corns = np.array(
        [[np.min(rays['ex']),np.min(rays['ey'])],
         [np.max(rays['ex']),np.max(rays['ey'])]])
    T_pxc_f = np.array(
        [[1*CELLS_PER_M, 0             , 0],
         [0            , -1*CELLS_PER_M, 0],
         [0            , 0             , 1]])
    pxc_corns=homp(T_pxc_f,f_corns)
    pxc_topleft = np.array([  np.floor(np.min(pxc_corns[:,0])), np.floor(np.min(pxc_corns[:,1]))  ]) #inclusive
    pxc_btmright= np.array([  np.floor(np.max(pxc_corns[:,0])), np.floor(np.max(pxc_corns[:,1]))  ]) + 1 #exclusive
    pxc_extent = pxc_btmright-pxc_topleft

    T_px_f = T_pxc_f.copy()
    T_px_f[:-1,-1] = -pxc_topleft

    px_corns=homp(T_px_f,f_corns)
    px_topleft = np.array([  np.floor(np.min(px_corns[:,0])), np.floor(np.min(px_corns[:,1]))  ]) #inclusive
    px_btmright= np.array([  np.floor(np.max(px_corns[:,0])), np.floor(np.max(px_corns[:,1]))  ]) + 1 #exclusive
    px_extent = px_btmright-px_topleft

    assert (np.all(pxc_extent == px_extent))

    if debug:
        print("fcorns\n",f_corns)

        for var in ['pxc_corns',
                    'pxc_topleft',
                    'pxc_btmright',
                    'pxc_extent',
                    'px_corns',
                    'px_topleft',
                    'px_btmright',
                    'px_extent',
                    ]:
            print(var+'\n', eval(var))
    return T_px_f, px_extent.astype('int64')

keep_for_window=(rays0['r']>0)*(rays0['r']<10)
with np.printoptions(formatter=formatter):
    T_px_f, px_extent = get_T_px_f_with_extent(rays[keep_for_window], debug=True)


Cell width: 0.05
fcorns
 [[      28.25       13.39]
 [      70.21       77.7 ]]
pxc_corns
 [[     564.97     -267.7 ]
 [    1404.27    -1554.07]]
pxc_topleft
 [     564       -1555   ]
pxc_btmright
 [    1405        -267   ]
pxc_extent
 [     841        1288   ]
px_corns
 [[       0.97     1287.3 ]
 [     840.27        0.93]]
px_topleft
 [       0           0   ]
px_btmright
 [     841        1288   ]
px_extent
 [     841        1288   ]


In [34]:
# convert to [start_cell_x,start_cell_y, d_cell, th] format
f_spt = np.vstack((rays['sx'].ravel(),rays['sy'].ravel())).T.copy()
f_ept = np.vstack((rays['ex'].ravel(),rays['ey'].ravel())).T.copy()
px_spt = homp(T_px_f, f_spt)
px_ept = homp(T_px_f, f_ept)
px_d = np.sqrt(np.sum((px_ept-px_spt)**2,axis=1))
px_th = np.mod(np.arctan2((px_ept-px_spt)[:,1], (px_ept-px_spt)[:,0]),2*np.pi)
def get_raster_coords(rays, T_px_f):
    # implicit inputs: T_px_f
    f_ept = np.vstack((rays['ex'].ravel(),rays['ey'].ravel())).T.copy()
    px_coords = homp(T_px_f, f_ept)
    px_coords = px_coords.reshape(rays['ex'].shape+(2,))
    return px_coords
# px_d[rays0['r']<0]=0
# %matplotlib widget
# plt.plot(rays0['th'][rays0['r']>0],px_th[rays0['r']>0],',')
th_spacing = 2*np.pi/360/4

outrays = np.vstack([px_spt.T,px_d,px_th/th_spacing])
%matplotlib widget
plt.plot(px_ept[:,0],px_ept[:,1],'b,')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
import numpy as np
th_spacing = 2*np.pi/360/4
th_bins = int(np.round(2*np.pi / th_spacing))
w_HIT = 1
w_MISS = 4 # Reduce to 1 if known no motion
bias = 0
wide_threshold = 30*np.pi/180
W, H = 800,1200
HIT = np.zeros((W,H,th_bins),'int16')
MISS = np.zeros((W,H,th_bins),'int16')
RFM = np.zeros((W,H,th_bins),'int8') # -1=TRANSPARENT, 0=UNK, 1=REFLECT
RFM_mr = np.zeros((W,H,th_bins),'int8')
WIDE = np.zeros((W,H,th_bins),'bool')
NARROW = np.zeros((W,H,th_bins),'bool')
prerender = np.zeros((W,H),'bool')
refl_cache = np.zeros((W,H),'int16')
trans_cache = np.zeros((W,H),'int16')
OCC = np.zeros((W,H),'int8')

def RFM_update_cell(x,y,th):
    # updates the RFM cell and the cached values 
    old_val = RFM[x,y,th]
    new_val = np.sign(w_HIT*HIT[x,y,th]-w_MISS*MISS[x,y,th])
    RFM[x,y,th] = new_val
    if old_val == -1:
        trans_cache[x,y] -=1
    if old_val == 1:
        refl_cache[x,y] -=1
    if new_val == -1:
        trans_cache[x,y] +=1
    if new_val == 1:
        refl_cache[x,y] +=1

def quantize(coords):
    return np.floor(coords).astype('int')
    
def quantize4(coords): # 4 pt xy antialiasing samples
    coords = np.array(coords)
    coords4 = (coords + (-0.5,-0.5,0.0), coords + (-0.5,0.5,0.0), 
               coords + (0.5,-0.5,0.0), coords + (0.5,0.5,0.0))
    return np.floor(coords4).astype('int')
    
def accumulate(rays):
    epsl = 1.
    # prerender lets us stop rendering a ray that would go through a HIT
    prerender[:,:]=0
    for ray in rays:
        # ray has a starting point, distance before it returned, and direction
        [x_start, y_start, d, th] = ray
        [slope_x, slope_y] = [cos(th*th_spacing), sin(th*th_spacing)]
    
        xyth = quantize4((x_start + d*slope_x, y_start + d*slope_y, th))
        prerender[xyth[:,0],xyth[:,1]]+=1
    
    for ray in rays:
        # ray has a starting point, distance before it returned, and direction
        [x_start, y_start, d, th] = ray
        [slope_x, slope_y] = [cos(th*th_spacing), sin(th*th_spacing)]
    
        for r in range(0,d-epsl):
            xyth = quantize((x_start + r*slope_x, y_start + r*slope_y, th))
            if not prerender[xyth[0],xyth[1]]:
                MISS[xyth[0],xyth[1],xyth[2]]+=1
                RFM_update_cell(xyth[0],xyth[1],xyth[2])
            else:
                break
        xyth = quantize4((x_start + d*slope_x, y_start + d*slope_y, th))
        HIT[xyth[:,0],xyth[:,1],xyth[:,2]]+=1
        for xyth_sample in xyth:
            RFM_update_cell(xyth_sample[0], xyth_sample[1], xyth_sample[2])
    
        
def classify_wide_narrow():
    WIDE[:,:]= refl_cache > wide_threshold
    WIDE &= refl_cache/trans_cache > wide_min_ratio
    NARROW[:,:] = (refl_cache>0) & (WIDE==False)

def remove_motion():
    RFM_mr[NARROW,:] = RFM[NARROW,:]-3 # map [-1,0,1]->[-4,-3,-2]
    RFM_mr[WIDE,:] = RFM[WIDE,:]
    stack = []
    seed_locs=np.argwhere(WIDE)
    for x,y in seed_locs:
        if (  NARROW[x-1,y-1] or NARROW[x-1,y] or NARROW[x-1,y+1] or
              NARROW[x,y-1] or NARROW[x,y+1] or
              NARROW[x+1,y-1] or NARROW[x+1,y] or NARROW[x+1,y+1]  ):
            thetas = RFM[x,y,:]
            for th in thetas:
                stack.append((x,y,th))
    while(stack):
        x,y,th = stack.pop
        t2d_neighbors = ((x-1,y-1),(x-1,y),(x-1,y+1),
                        (x,y-1),  (x,y)  ,(x,y+1),
                        (x+1,y-1),(x+1,y),(x+1,y+1))
        for x,y in t2d_neighbors:
            if not NARROW(x,y): continue
            for th in (th-1, th, th+1):
                if RFM_mr[x,y,th] == -2: # Unvisited, NARROW, REFL
                    RFM_mr[x,y,th] = 4
                    stack.append((x,y,th))
    # codes: -4:NAR,TRANS -3:NAR, UNK -2:NAR,DYN 
    #        -1:WIDE,TRANS 0:WIDE,UNK  1:WIDE,STATIC
    #         4:NAR,CON
    RFM_mr[NARROW,:] //= 4 # remap -4,-3,-2,4 -> -1,0,0,1; deleting motion
    # Classify occupancy: -2=FREE, -1=DYNAMIC, 0=UNK, 1=STATIC
    OCC[:,:] = 0 
    OCC[trans_cache>0] = -2 # seen through defaults FREE
    OCC[NARROW] = -1 # narrow defaults DYNAMIC
    OCC[WIDE] = 1
    # The following line is an optimization of
    # OCC[np.any(RFM_mr==1,axis=1)] = 1
    OCC[NARROW] += np.any(RFM_mr[NARROW,:]==1,axis=1)*2
    OCC[NARROW & (transcache==0)] = 1 # Default uncontested cells to STATIC

def classify_reflections(rays):
    is_reflection = []
    epsl = 1.999

    for ray in rays:
        [x_start, y_start, d, th] = ray
        [slope_x, slope_y] = [cos(th*th_spacing), sin(th*th_spacing)]
    
        for r in range(0,d-epsl):
            xyth = quantize((x_start + r*slope_x, y_start + r*slope_y, th))
            if OCC[xyth[0], xyth[1]]:
                is_reflection.append(True)
                break
        else:
            is_reflection.append(False)
    return is_reflection

def process_scan(rays):
    # This is the do-everything main function
    accumulate(rays)
    classify_wide_narrow()
    remove_motion()
    # Now the RFM estimate without motion removal is in RFM
    # and the version with the motion removed is in RFM_mr.
    # OCC holds the estimated occupancy classification
    
    # classify_rays() can be used to check rays for reflection in
    # another thread, and build a new RFM without them

# Basic RFM
The following calculates the basic RFM, accumulating HITs and MISSes. \
It does not account for reflections, and doesn't link up objects (we do that later)

Typically runs ~80 scans/second

In [36]:
loclip = np.array([0,0,0]).reshape(3,1)
hiclip = (np.array(HIT.shape)-1).reshape(3,1)
def quantize(coords):# 3xN
    return np.clip(np.floor(coords),loclip,hiclip).astype('int')

def quantize4(coords): # 4 pt xy antialiasing samples # 3xN
    coords = np.array(coords).T  # Nx3
    aa = np.array(((-0.5,-0.5,0.0), (-0.5,0.5,0.0), 
                  ( 0.5,-0.5,0.0), ( 0.5,0.5,0.0) )) # 4x3
    coords4 = aa + coords[:,np.newaxis,:] # N,4,3
    coords4= coords4.reshape(-1,3).T
    return np.floor(np.clip(coords4,loclip,hiclip)).astype('int')

def RFM_update_cell(x,y,th):
    #TODO Write me!
    pass
    
# Define the accumulate operator, that renders rays
def accumulate(rays, notref):
    epsl = 1.
    # prerender lets us stop rendering a ray that would go through a HIT
    prerender[:,:]=0
#     for ray in rays:
    # ray has a starting point, distance before it returned, and direction
    [x_start, y_start, d, th] = rays
    [slope_x, slope_y] = [np.cos(th*th_spacing), np.sin(th*th_spacing)]

    xyth = quantize4((x_start[notref] + d[notref]*slope_x[notref], y_start[notref] + d[notref]*slope_y[notref], th[notref]))
    prerender[xyth[0],xyth[1]]=1
    
#     for ray in rays:
    # ray has a starting point, distance before it returned, and direction
    [x_start, y_start, d, th] = rays
    [slope_x, slope_y] = [np.cos(th*th_spacing), np.sin(th*th_spacing)]
    
    
    
    live=np.ones_like(d)
    r=np.full_like(d,0)
    for r0 in range(0,200):
#         print(r0, end=' ')
        r[:]=r0
        live[r0>=d-epsl]=0
#         print(np.sum(live))
        r[live==0]=100000
        xyth = quantize((x_start + r*slope_x, y_start + r*slope_y, th))
#         if r0>1
        live*=(prerender[xyth[0],xyth[1]]==0)
        MISS[xyth[0],xyth[1],xyth[2]]+=1
        RFM_update_cell(xyth[0],xyth[1],xyth[2])
    xyth = quantize((x_start[notref] + d[notref]*slope_x[notref], y_start[notref] + d[notref]*slope_y[notref], th[notref]))
    HIT[xyth[0],xyth[1],xyth[2]]+=1
    RFM_update_cell(xyth[0], xyth[1], xyth[2])
print(outrays.shape)
arays=outrays.copy()
# arays[0]-=2300
# arays[1]-=1500
ls=0
keep=(rays0['r']<10)*(rays0['r']>0)
CHUNK_SIZE = 1081
for s in tqdm(range(CHUNK_SIZE,arays.shape[1]+1,CHUNK_SIZE)):
    
#     print(int(s/arays.shape[1]*1000),end=' ')
    tmprays=arays[:,ls:s][:,keep[ls:s]]
    accumulate(tmprays,np.ones_like(tmprays[0],dtype=np.bool))
    ls=s

## Draw Figures
The following calculates the basic RFM in bulk (Faster in Python and good for prototyping).
Then we take a slice out though the glass to make the figure.

We don't discuss it in the paper, due to lack of space, but this also shows several other visualizations good for exploring the RFM


In [73]:
# Specialized debug cell
%matplotlib widget
disco=HIT/(np.float32(0.1)+HIT+MISS)
RFM=HIT*w_HIT-MISS*w_MISS
sud=HIT+MISS
# del disco
del sud
# del disco
plt.figure(1)
ax=plt.subplot(221)
# plt.imshow(np.clip(np.max((HIT),axis=2).T,-3,15))
plt.imshow(np.clip(np.max((HIT),axis=2).T,-10,10))
plt.title('Max HIT [0,10]')
plt.subplot(222,sharex=ax,sharey=ax)
mean_active = (np.sum(MISS,axis=2,dtype=np.float32)
              /
              (np.sum(MISS>0,axis=2)+np.float32(.0001)))[:,:,np.newaxis]
plt.imshow(np.clip(mean_active[:,:,0].T,-1,10))
plt.title("Average RFM probes [0,10]")
plt.subplot(223,sharex=ax,sharey=ax)
plt.imshow(np.clip(np.max(HIT/(mean_active+0.01),axis=2).T,-1,3))
plt.title('Normalized Max HIT')

plt.subplot(224,sharex=ax,sharey=ax)
plt.imshow(np.clip(np.max(HIT/(mean_active+0.01),axis=2).T,-1,3))
plt.title('Example RFM Slice Line')
plt.plot([340,360],[142,142],'r')

del mean_active
# # plt.imshow(prerender.T)
# plt.figure(2)
# plt.plot(np.max(np.max(HIT,axis=0),axis=0))
# plt.plot(HIT[253,212]*1000.0)
# plt.plot(-MISS[253,212]*1000.0)
# plt.plot(HIT[235,418]*1000.0) #down
# plt.plot(HIT[287,539]*1000.0) #left
# plt.plot(HIT[290,493]*1000.0) #left
# plt.plot(HIT[286,552]*1000.0) #left
# plt.plot(HIT[227,552]*1000.0) #up
# # plt.plot(rays[0],rays[1],',')
# # x = plt.ginput(3)


# plt.plot(np.sum(np.sum(HIT,axis=0),axis=0))


plt.figure(4)
plt.title("H Shapes Figure, Option 1")
# Need to take max of neighboring slices due to aliasing
plt.imshow(np.clip(np.max(RFM[:,850:852,:],axis=1).T,0,2),aspect='auto') 
plt.xlim(320,540)
# plt.ylim(0,700)
# plt.imshow(np.clip(np.sum(disco[:,:,:],axis=0).T,0,2))
plt.xlabel('x [1 unit = 5cm]')
plt.ylabel('theta [720 units = 180deg]')

plt.figure(5)
plt.title("H Shapes Figure,Option 2")
plt.imshow(np.clip(np.max(RFM[:,142:143,:],axis=1).T,-2,2),aspect='auto',cmap='bwr_r')
plt.xlabel('x [1 unit = 5cm]')
plt.ylabel('theta [720 units = 180deg]')
plt.xlim(295,425)


# plt.figure(5)
# plt.imshow(np.clip(disco[:,829,:].T,0,10))
# plt.figure(6)
# plt.imshow(np.clip(HIT[:,830,:].T,0,10))
# plt.figure(7)
# plt.imshow(np.clip(HIT[:,831,:].T,0,10))
# plt.figure(8)
# plt.imshow(np.clip(HIT[:,832,:].T,0,10))
# plt.figure(9)
# plt.imshow(np.clip(HIT[:,833,:].T,0,10))
# plt.figure(11)
# plt.imshow(np.clip(HIT[:,851,:].T,0,10))
# plt.imshow(np.clip(HIT[:,:,736-360].T,0,10))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(295, 425)

## Figure captions
*Option 1:* Glass panes are easily detectable in the Reflectance Field Map by their distinctive H shape. 
Shown here is a slice through the RFM along a wall made of several panes of glass, producing the repeated H patterns seen here.
The arms of the H are formed by the edges of the glass or its frame which scatter light in many angles,
and the cross bar is caused by the smooth face of the glass reflecting mostly perpendicularly.
Note that despite the wide variation in visibility between different pieces of glass, and the gaps 
in some of the H arms due to occlusions, this wall is reliably detectable at 91deg = 364 units 

[TODO: figure out how to make Figure scale show degrees]

Unlike intensity or multiecho methods, our's does not require a bright specular peak, or an object behind to be detected.
And unlike Foster et al 2013[Cite], the gaps and secondary detection angles shown here do not confuse our detector.
Even an occluder directly blocking the view of the glass only causes the loss of the blocked part.

*Option 2:*: Similar text to Option 1, but mention Blue is reflective, Red is transparent, and white is areas of the 
reflectance field not probed, due to occlusions or the sensor path. \
Option two is made available to demostrate ability to show alternate color scheme, an alternate target with more regular 
(but less interesting!) pieces of glass, 

In [12]:
# Detect Reflections
REF_BLOCK_THRESH = 0.3
def classify_cells_bulk():
    # Justification: mean_active is an approximation of how many times we should have seen a cell.
    # It is generated by looking at the mean of HIT+MISS for cells that have ever had either, meaned along the theta axis
    # Note that if there is a huge variation in the HIT+MISS sum, this could be inaccurate(i.e. if we take different paths 
    # near the point.)
    # The score is how many times we saw/expected attempts. We take the max score over angles to represent the most opaque
    # direction. Scores significantly above 1.0 indicate a bad condition
    #
    fullcount=MISS+HIT
    mean_active = (np.sum(fullcount,axis=2,dtype=np.float32)
                  /
                  (np.sum(fullcount>0,axis=2)+np.float32(.0001)))[:,:,np.newaxis]
    score = np.max(HIT/(mean_active+np.float32(.0001)),axis=2)
    refl_cache[1:-1,1:-1]=(score>REF_BLOCK_THRESH)[1:-1,1:-1]
    return score
classify_cells_bulk()

array([[ 0.,  0.,  0., ...,  0.,  0., -0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [-0.,  0.,  0., ...,  0.,  0., -0.]])

In [13]:
%matplotlib widget
plt.imshow(refl_cache.T)
plt.title('Cells that stop rays for reflectance calculations')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Cells that stop rays for reflectance calculations')

# Classify Reflection Rays
Mark the depths where rays try to go through another object. 
PEN_DEPTH controls this. We set to 30cm to deal with the stupid 20cm short detections that seem to plague our laser.

The basic idea is that all rays render MISSes, but only rays that travel less than PEN_DEPTH from the surface can render HITs

In [14]:
# classify reflectionness of a ray and clip to a given penetration depth
PEN_DEPTH = np.ceil(0.3*CELLS_PER_M)
def classify(rays):
    pen_depth = PEN_DEPTH
    # prerender lets us stop rendering a ray that would go through a HIT
    prerender[:,:]=0
    
#     for ray in rays:
    # ray has a starting point, distance before it returned, and direction
    [x_start, y_start, d, th] = rays
    [slope_x, slope_y] = [np.cos(th*th_spacing), np.sin(th*th_spacing)]
    
    
    notref=np.ones_like(d,dtype=bool)
    r=np.full_like(d,0)
    for r0 in tqdm(range(0,200)):
#         print(r0,end=', ')
        r[:]=r0
        xyth = quantize((x_start + r*slope_x, y_start + r*slope_y, th))
#         if r0>1
        stopped=(refl_cache[xyth[0],xyth[1]]==1)
        notrefish = r0 > d[stopped]-pen_depth
        notref[stopped]*=notrefish
#         print(r0+pen_depth)
        d[stopped]=np.minimum(d[stopped],r0+pen_depth)
    rays = np.vstack([x_start, y_start, d, th])
    return rays, notref

arays=outrays.copy()

arays,notref = classify(arays)
print("classified")

#
# The returned rays now have distances reflecting how far to render them. 
# If the distance is less than the original, we want to not render the HIT when we rerender

100%|██████████| 200/200 [00:21<00:00,  9.40it/s]

classified


In [47]:
Hstore=[]
Mstore=[]

# Show Stabiliation of Reflection Removal:

In [43]:
%matplotlib widget

from tqdm import tqdm
plt.imshow(refl_cache.T)

for refitr in range(5):
    Hstore.append(HIT)
    Mstore.append(MISS)
    
    classify_cells_bulk()
    
    arays=outrays.copy()
    arays,notref = classify(arays)

    HIT = np.zeros((W,H,th_bins),'int16')
    MISS = np.zeros((W,H,th_bins),'int16')
    ls=0
    keep=(rays0['r']<30)*(rays0['r']>0)
    CHUNK_SIZE = 1081

    for s in tqdm(range(CHUNK_SIZE,arays.shape[1]+1,CHUNK_SIZE)):
        tmprays=arays[:,ls:s][:,keep[ls:s]]
        nrtemp=notref[ls:s][keep[ls:s]]
        accumulate(tmprays, nrtemp)
        ls=s
        
    plt.figure()
    plt.imshow(refl_cache.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

100%|██████████| 2775/2775 [00:35<00:00, 78.30it/s]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

100%|██████████| 2775/2775 [00:36<00:00, 76.01it/s]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

100%|██████████| 2775/2775 [00:35<00:00, 77.56it/s]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

100%|██████████| 2775/2775 [00:35<00:00, 77.92it/s]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
# Visualize the 2d occupancy score
%matplotlib widget
score=classify_cells_bulk()
plt.imshow(np.clip(score,-5,5).T)
plt.figure()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [45]:
#Visualize all nearest points
%matplotlib widget

r=rays0['r'].copy()
r=r.reshape(-1,1081)
r[r<=0]=100
m=np.argmin(r,axis=1)

r0=rays0.reshape(-1,1081)
print(m.shape)
mr=np.take_along_axis(r0,m[:,np.newaxis],axis=1)
print(mr)
plt.plot(mr['ex'],mr['ey'],',')

In [62]:
HIT = np.zeros((W,H,th_bins),'int16')
MISS = np.zeros((W,H,th_bins),'int16')
ls=0
keep=(rays0['r']<30)*(rays0['r']>0)
arays=outrays.copy()

r=arays[2]
r=r.reshape(-1,1081*3)
m=np.min(r,axis=1)
r*=0
r[:]=m[:,np.newaxis]

CHUNK_SIZE = 1081*10
print(notref)
for s in tqdm(range(CHUNK_SIZE,arays.shape[1]+1,CHUNK_SIZE)):
    tmprays=arays[:,ls:s][:,keep[ls:s]]
    nrtemp=np.zeros(tmprays.shape[1],np.bool)
    accumulate(tmprays, nrtemp)
    ls=s
    

[ True  True  True ...  True  True  True]


100%|██████████| 277/277 [00:19<00:00, 14.11it/s]


In [65]:
%matplotlib widget
plt.imshow(np.clip(np.max((MISS),axis=2).T,-1,1)*.5+np.clip(np.any((Hstore[-1]>Mstore[-1]),axis=2).T,-1,1))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …